# NBME - Score Clinical Patient Notes
![](https://storage.googleapis.com/kaggle-competitions/kaggle/33607/logos/header.png)

今回提示されているデータは米国の医師国家試験であるUSMLE Step2 Clinical Skills examination（日本ではPost CC OSCEなどの試験に相当します）から抽出されたものです．この試験では，医学部実習生が模擬患者との医療面接において適切な臨床情報を取り出す能力を測ります．
各受験者は，まず模擬患者（症例を実際の患者さんのように提示するよう訓練された方々）の診察を行います．その後，受験者はカルテに診察で得られた重要事項を書き込みます．そのカルテは経験ある医師が問題症例と関連した重要事項や特徴（おそらく症状や検査所見のこと）を見て採点基準に従って採点されます．

今回のコンペの目標は，模擬患者の診察から得られた情報が記載されたカルテの既往歴・現病歴(patient history)に着目して各カルテの適切事項を自動で識別する方法を確立することです．

<div class="alert alert-block alert-warning">
<b>Terms:</b>
<ul>
    <li><b>Clinical cases</b>：模擬患者が受験者（医学生，研修医，医師）に提示するシナリオ（症状，主訴など）．今回のデータセットでは10症例提示されている．</li>
    <li><b>Patient Note</b>：診察（身体診察，問診）で得られる患者さんに関する重要情報について書かれた文.おそらくカルテに相当する．</li>
    <li><b>Feature</b>：臨床的に重要な概念，事項．採点基準において各症例における重要情報について説明されている．</li>
</ul>
</div>

### References:
- [NBME - Score Clinical Patient Notes](https://www.kaggle.com/ruchi798/nbme-score-clinical-patient-notes)

## About Training Data
- `patient_notes.csv` - 約40,000症例のカルテ（病歴に関する部分）． これらの一部しかアノテーションされていない．アノテーションされていないカルテについては教師なし学習を適用するとよい．テストセットのカルテはこのファイルの公開版には含まれていない．
 - `pn_num` - 各カルテの識別子
 - `case_num` - 各カルテで書かれている臨床症例の識別子
 - `pn_history` - 受験者によって記録された医療面接の文
- `features.csv` - 各症例における重要事項に関する採点基準
 - `feature_num` - 各重要事項・特徴の識別子
 - `case_num` -　各臨床症例の識別子
 - `feature_text` - 重要事項に関する説明
- `train.csv` - 1000個のカルテに対する特徴アノテーション（10症例$\times$100)
 - `id` - カルテと特徴の組み合わせの識別子
 - `pn_num` - この行でアノテーションされたカルテ
 - `feature_num` - この行でアノテーションされた特徴
 - `case_num` - このカルテがどの症例のものか
 - `annotation` - カルテの中で特徴を示す部分．一つのカルテ内にその特徴を示す部分が複数ある可能性もある．
 - `location` - カルテ内の各特徴の場所を示す**文字区間**．区間がセミコロンで区切られている場合は，複数区間がアノテーションを示すのに必要である．

<div class="alert alert-block alert-info" style="font-size:14px; line-height: 1.7em;">
     &nbsp;<b><u>The definition of character span: 文字区間の定義</u></b>
 
* 今回，予測することとなるのが，まさにこの<b>文字区間</b>です。 
* 文字区間は文章中の特定の範囲を示すためのインデックスのペアのことを指します。
    * 例えば，i jという区間は<b>インデックスi~jの範囲(ただし，iは含んでjは含まない)</b>のことを表します。
    * pythonでは<b>スライス[i:j]</b>に相当します。 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from termcolor import colored
from ast import literal_eval
import plotly.express as px


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
FEATURES_CSV = '../input/nbme-score-clinical-patient-notes/features.csv'
PN_CSV = '../input/nbme-score-clinical-patient-notes/patient_notes.csv'
SAMPLE_SUB_CSV = '../input/nbme-score-clinical-patient-notes/sample_submission.csv'
TEST = '../input/nbme-score-clinical-patient-notes/test.csv'
TRAIN = '../input/nbme-score-clinical-patient-notes/train.csv'

In [ ]:
train = pd.read_csv(TRAIN)
print(train.shape)
train.head()

In [ ]:
train.info()

In [ ]:
features_csv = pd.read_csv(FEATURES_CSV)
features_csv.head()

In [ ]:
len(features_csv['feature_num'].unique())==len(features_csv['feature_num'])

In [ ]:
pn_csv = pd.read_csv(PN_CSV)
pn_csv.head()

In [ ]:
len(pn_csv['case_num'].unique())

## Patient Note Overview

In [ ]:
def loc_list_to_ints(loc_list):
    to_return = []
    for loc_str in loc_list:
        # 文字区間が複数ある場合セミコロで区切られているので、それで分割してリストへ
        loc_strs = loc_str.split(";")
        # 各文字区間について以下の操作を行う
        for loc in loc_strs:
            # 文字区間は最初と最後の文字のインデックスのペア
            start, end = loc.split()
            # 文字区間をタプルとして格納
            to_return.append((int(start), int(end)))
    return to_return

def annotation(pn_num):
    print('\033[31m'+'Annotation '+ '-'*55+'\033[0m')
    subset = train[train['pn_num'] == 16]
    location_lst = subset.location
    for f_num, i in enumerate(location_lst):
        loc_list = literal_eval(i)
        loc = loc_list_to_ints(loc_list)
        print('\033[32m'+f'feature_num:{f_num}'+'\033[0m')
        if len(loc) > 0:
            for j in loc:
                pn = pn_csv[pn_csv['pn_num']==16]['pn_history'][16]
                print(pn[j[0]:j[1]])
        else:
            print('->>no annotation')

In [ ]:
def pn_overview(pn_num):
    case_num = pn_csv[pn_csv['pn_num']==pn_num]['case_num'].iat[0]
    print('\033[31m'+f'\nPatient Notes (History)：{pn_num}'+'\033[0m')
    print('\033[31m'+f'\nCase Number：{case_num} '+'-'*50+'\033[0m')
    print(pn_csv[pn_csv['pn_num']==pn_num]['pn_history'][pn_num])
    print('\033[31m'+'\nFeatures ' + '-'*55+'\033[0m')
    print(features_csv[features_csv['case_num'] == case_num]['feature_text'])

In [ ]:
pn_overview(16)
annotation(16)

各カルテでどのような内容が重要な臨床事項になるのかを`pn_num`が16のカルテを例に見てみます．<br>
- **Patient NOTE**：
17歳男性，動悸を主訴に来院．3～4ヶ月間続く間欠性の「どきどき感」を訴えている．2日前にサッカーの試合中に症状の発現がみられたが，意識が遠のくような感じがしたらしい．（意識の消失はなかった）特記事項：患者は勉強のためのアデロールの乱用を認めている（週に1~3回程度）．最近のサッカーの試合の前には，前日の夜と当日の朝にアデロールを服用した．息切れ，発汗，発熱，寒気，頭痛，倦怠感，視覚・聴覚の変化，腹痛，排便・排尿の変化は認められない．
 - 既往歴：なし
 - 服薬歴：友達のアデロールを服用
 - 家族歴：母親が甲状腺疾患，父親が最近狭心症になった
 - ワクチン接種歴：スケジュール通りにワクチン接種済み
 - 社会歴：大学1年生，週に3回（週末）3~4杯飲酒，喫煙歴なし，マリファナ使用歴あり，性活動は活発，コンドームは使用
<br>

`pn_num`：16では症例0の問題を診察しており，以下の12項目が重要な臨床事項であるようです．
- **Features**
    - 心筋梗塞の家族歴
    - 甲状腺疾患の家族歴
    - 間欠性の症状
    - めまい
    - 頭髪の変化なし，爪の変化なし，暑がり，寒がりなどの症状なし：甲状腺疾患では，多汗，暑がりなど温度に対する不耐性や爪の変化が生じることがあるのでそれを意識しての項目でしょう．
    - アデロールの使用あり：アデロールはADHD(注意欠如多動性障害)と睡眠発作の治療に使われるアンフェタミン製剤です．
    - 息切れ
    - カフェインの使用あり
    - 動悸
    - 数ヶ月の持続期間
    - 17歳
    - 男性

In [ ]:
pn_overview(18)

## Distribution of features & patient notes of each cases
各症例のfeature数とカルテ数を確認してみます。

In [ ]:
feature_counts = features_csv.groupby("case_num").count()
fig = px.bar(data_frame =feature_counts, 
             x = feature_counts.index,
             y = 'feature_num')
fig.show()

In [ ]:
pn_counts = pn_csv.groupby("case_num")['pn_num'].count()
fig = px.bar(data_frame = pn_counts, 
             x = pn_counts.index,
             y = 'pn_num')
fig.show()

各カルテについてannotaionされたfeatureの数と採点基準となるfeatureの数が異なるカルテが存在するのか調べてみると，0でした．

In [ ]:
feature_count_per_pn = pd.DataFrame()
feature_count_per_pn['feature_count'] = train.groupby('pn_num')['feature_num'].count()

rubric_feature_count = []
for i in feature_count_per_pn.index:
    case_num = train[train['pn_num']==i]['case_num'].iat[0]
    rubric_feature_count.append(len(features_csv[features_csv['case_num']==case_num]))
feature_count_per_pn['rubric_feature_count'] = rubric_feature_count

cnt = 0
for i in feature_count_per_pn.index:
    if feature_count_per_pn.at[i, 'feature_count'] != feature_count_per_pn.at[i, 'rubric_feature_count']:
        cnt += 1
cnt

## About Evaluation
今回，予測するのは前述の通り，文字区間です。<br>
予測指標となるのは，[micro-averaged F1 score](https://scikit-learn.org/stable/modules/model_evaluation.html#from-binary-to-multiclass-and-multilabel)です。この指標は，TP(真陽性),FN(偽陰性)，FP(偽陽性)の値から算出されます。
<br>
例えば，

| ground-truth | prediction |
|--------------|------------|
| 0 3; 3 5     | 2 5; 7 9; 2 3 |

の場合，真の正解と予測値の文字区間は以下のようになります。

| ground-truth | prediction |
|--------------|------------|
| 0,1,2,3,4     | 2,3,4,7,8 |

この場合，TP(ground-truthとpredictionどちらにも含まれる)，FN(ground-truthのみに含まれる)，FP(predictionのみに含まれる)の個数は以下のようになり，これらの値から最終的なF1値が算出されることになります。

|  | TP | FN | FP |
|----------|--------------|------------|-----------|
| index | 2,3,4 | 0,1 | 7,8 |
| count | 3 | 2 | 2 |